In [49]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
global batchSize
batchSize = 200
global testSize
testSize = 50
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})

#     Y = tf.expand_dims(tf.cast(tf.argmax(v_ys,1), tf.float32),1)

    result1 = sess.run(error, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#     result2 = sess.run(Y, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result1
def evaluate(v_xs):
    predictedVal = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    return predictedVal

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d2(x, W): # stride 2x2
    return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='VALID')

def conv2d0(x, W): # no stride
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x): # Not needed
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 13200]) # 28x28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 66, 200, 1])
# udacity image size : 66x200

## need to add normalization here

## conv1 layer ##
W_conv1 = weight_variable([5,5,1,24]) # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([24])
h_conv1 = tf.nn.relu(conv2d2(x_image, W_conv1) + b_conv1) # feature map @ layer 1 size 31x98

## conv2 layer ##
W_conv2 = weight_variable([5,5, 24, 36]) # patch 5x5, in size 24, out size 36
b_conv2 = bias_variable([36])
h_conv2 = tf.nn.relu(conv2d2(h_conv1, W_conv2) + b_conv2) # feature map @ layer 2 size 14x47

## conv3 layer 
W_conv3 = weight_variable([5,5, 36, 48]) # patch 5x5, in size 36, out size 48
b_conv3 = bias_variable([48])
h_conv3 = tf.nn.relu(conv2d2(h_conv2, W_conv3) + b_conv3) # feature map @ layer 3 size 5x22 (22= ceiling((47-4)/2))

## conv4 layer 
W_conv4 = weight_variable([3,3, 48, 64]) # patch 5x5, in size 48, out size 64
b_conv4 = bias_variable([64])
h_conv4 = tf.nn.relu(conv2d0(h_conv3, W_conv4) + b_conv4) # feature map @ layer 4 size 3x20

## conv5 layer 
W_conv5 = weight_variable([3,3, 64, 64]) # patch 5x5, in size 32, out size 64
b_conv5 = bias_variable([64])
h_conv5 = tf.nn.relu(conv2d0(h_conv4, W_conv5) + b_conv5) # feature map @ layer 5 size 1x18



## flatten ##
# [n_samples, 1, 18, 64] ->> [n_samples, 1*18*64]
h_conv5_flat = tf.reshape(h_conv5, [-1, 1*18*64])

## fully_connected_layer 1 ##
W_fc1 = weight_variable([1*18*64, 100])
b_fc1 = bias_variable([100])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fully_connected_layer 2  ##
W_fc2 = weight_variable([100, 50])
b_fc2 = bias_variable([50])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

## fully_connected_layer 3  ##
W_fc3 = weight_variable([50, 10])
b_fc3 = bias_variable([10])
h_fc3 = tf.matmul(h_fc2, W_fc3) + b_fc3

## fully_connected_layer 4  ##
W_fc4 = weight_variable([10, 1])
b_fc4 = bias_variable([1])
prediction = tf.nn.relu(tf.matmul(h_fc3, W_fc4) + b_fc4)

Y = tf.expand_dims(tf.cast(tf.argmax(ys,1), tf.float32),1)
#===========
cost = tf.reduce_sum(tf.pow(prediction - Y, 2))/(batchSize)
error = tf.reduce_sum(tf.pow(prediction - Y, 2))/(testSize)
# Gradient descent
train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(cost)
#======


sess = tf.Session()
# important step
sess.run(tf.initialize_all_variables())

for i in range(500000):
    batch_xs, batch_ys = mnist.train.next_batch(batchSize)
    batch_xs_new = np.zeros((batchSize,13200),dtype=np.float32)
    for j in range(batchSize):
        batch_xs_new[j,:] = np.append(batch_xs[j,:],np.zeros(12416))
#     print 'start training current batch'
    sess.run(train_step, feed_dict={xs: batch_xs_new, ys: batch_ys, keep_prob: 0.7})
#     print 'finished training current batch'
    if i % 200 == 0:
#         testImg = mnist.test.images[:testSize,:]
#         testLabels = mnist.test.labels[:testSize,:]
        testImg,testLabels = mnist.train.next_batch(testSize)
        newTestImg = np.zeros((testSize,13200),dtype=np.float32)
        for j in range(testSize):
            newTestImg[j,:] = np.append(testImg[j,:], np.zeros(12416))
        res1 = compute_accuracy(newTestImg, testLabels)
        print 'accuracy at step',i,':',res1
        dataInput = newTestImg[0:1,:];
        dataOutput = testLabels[0,:];
        currResult = evaluate(dataInput)
        print 'value input is',dataOutput,'predict is',currResult

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
accuracy at step 0 : 26.3238
value input is [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.] predict is [[ 0.63748765]]
accuracy at step 200 : 5.50326
value input is [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.] predict is [[ 4.35206985]]
accuracy at step 400 : 6.11511
value input is [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.] predict is [[ 4.72980928]]
accuracy at step 600 : 9.03751
value input is [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.] predict is [[ 3.7547307]]
accuracy at step 800 : 5.44205
value input is [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.] predict is [[ 2.03882527]]
accuracy at step 1000 : 5.31563
value input is [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.] predict is [[ 3.18867111]]
accuracy at step 1200 : 3.34079
value input is [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.] predict is [[ 6.40247631]]
ac

KeyboardInterrupt: 

In [12]:
newTestImg.shape

(13200,)